#### Analytical Solution

For a board of length D, we need to move forward D times. And the rest of the moves are stays. We will have N-D of those.

So we can view this as a combination. Of the N moves, we have to pick D to the forwards. So that's N-Combination-D, which is **N!/(D!(N-D)!)**

It can also be viewed as a permutation of N moves, where D are of type F and (N-D) of stays. So it's permutations with repetitions: N!/(D!(N-D)!). Which is the same thing as before.

In [5]:
import numpy as np
def fac(n):
    return np.prod(range(1,n+1))

In [6]:
def ways(n,d):
    return fac(n)/(fac(n-d)*fac(d))

In [7]:
ways(8,5)

56

#### Recursive Solution

For recursion, we just have to figure out the first step. In this case, we can either stay or go. If we stay, we have N-1 moves and D positions. If we go forward, we have N-1 moves and D-1 postions. The total solutions is the sum of the two options. So we'll just sum them up.

For every recursion problem, the steps we take are the same 4.

    1) What is the first step (in this case: go forward or stay)
    2) If we stay, D remains the same, if we go, D is reduced by 1. In either case, N is reduced by 1.
    3) How do we combine the pieces (in this case: sum the two cases)
    4) What are the end conditions (in this case: if D>N then 0, if D=1 then N)
    
Below I have two versions with different end conditions. If D=1 then N or If D=N then 1. Either of these is enough. But you can use all of them if you so choose.

In [8]:
def ways2(n,d):
    if n<d or d<0:
        return 0
    elif n==d:
        return 1
    else:
        return ways2(n-1,d)+ways2(n-1,d-1)

In [9]:
ways2(,5)

56

In [10]:
def ways2(n,d):
    if n<d or d<0:
        return 0
    elif d==1:
        return n
    else:
        return ways2(n-1,d)+ways2(n-1,d-1)

In [11]:
ways2(8,5)

56

#### Memoization

The function above calls itself twice. This leads to exponential number of calls. One way to fix it is to do memoization. We just create a dictionary. We build a tuple from the input parameters, and that tuple is the key to the dictionary. If a value exists for the key, we return that. And if not, then we compute it, update the dictionary and the return the computed value.

In [3]:
from collections import defaultdict
dic = defaultdict(int)
def ways2(n,d):
    if n<d or d<0:
        return 0
    elif d==1:
        return n
    elif dic[(n,d)]:
        return dic[(n,d)]
    else:
        dic[(n,d)]=ways2(n-1,d)+ways2(n-1,d-1)
        return dic[(n,d)]

In [12]:
ways2(8,5)

56

#### Dynamic Programming

Another approach is to just do it using loops. We follow the same formulation as recursion. But instead of calling itself, we just use the previously computed array. Consider this an alternate step to memoization.

In [10]:
import numpy as np
w = np.zeros((10,10))
def ways3(n,m):
    for i in range(1,n+1):
        w[i][1]=i
        for j in range(2,i+1):
            w[i][j]=w[i-1][j]+w[i-1][j-1]
    return w[n][m]

In [11]:
ways3(8,5)

56.0

A nice things about using Dynamic Programming is that, we can easily look up the computed values. Look at the array below. Each row is the number of moves. And Each column is the positions on the board. So each value is the sum of the one above it and the one above-and-left.

We can also clearly see the end conditions. If M=1 then N. If M=N then 1.

In [12]:
w

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   2.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   3.,   3.,   1.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   4.,   6.,   4.,   1.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   5.,  10.,  10.,   5.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   6.,  15.,  20.,  15.,   6.,   1.,   0.,   0.,   0.],
       [  0.,   7.,  21.,  35.,  35.,  21.,   7.,   1.,   0.,   0.],
       [  0.,   8.,  28.,  56.,  70.,  56.,  28.,   8.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

#### Extension

Now let's extend the problem to be this. At each move, in addition to going forward one or staying, you can also go backward by one (but can't go outside the board).

Let's see what happens to the various solutions.

**Analytical:** Now the solutions have three options, and that makes the combinatorics more difficult. But we can still solve it if we make the board limitless.

We need M forward moves. Of the remaining N-M moves, (N-M)//2 can be backward moves (so that we can have as many forwards moves to compensate). The rest will be stays. So we can loop over from 0 to (N-M)//2. For each, we'll know the number of B, F and S moves. We just need all permutation of this, and the formula is N!/B!F!S! (total factorial by the factorial of each repeated count).

In [19]:
def ways3(n,m):
    w = 0
    for b in range((n-m)//2+1):
        w += int(fac(n)/(fac(b)*fac(m+b)*fac(n-m-2*b)))
    return w

In [20]:
ways3(8,5)

112

**Recursion:** In this case, we just add an extra term to represent the backward step. The end conditions become a bit more detailed, but that's about it.

In [21]:
def ways3(n,m):
    if n<0:
        return 0
    elif n==0 and m==0:
        return 1
    elif n==1 and abs(m)<=1:
        return 1
    if n<abs(m):
        return 0
    else:
        return ways3(n-1,m)+ways3(n-1,m-1)+ways3(n-1,m+1)

In [22]:
ways3(8,5)

112

But the true problem is for a finite board that we can't go past in either direction. Solving that analytically is difficult. But with recursion, it's just updating the end conditions. To do this, we need to also pass a global_d to the function, since we keep modifying and reducing d in the subsequent functions that we call. If d is less than 0 or greater than global_d, the function will return 0.

In [23]:
def ways3(n,d,global_d):
    if d<0 or d>global_d:
        return 0
    if n<0:
        return 0
    elif n==0 and d==0:
        return 1
    elif n==1 and d<=1:
        return 1
    if n<d:
        return 0
    else:
        return ways3(n-1,d,global_d)+ways3(n-1,d-1,global_d)+ways3(n-1,d+1,global_d)

In [24]:
ways3(8,5,5)

96

In the code above, we have changed the input parameters of the code (because we added a third input). That can be avoided by using a default value for global_d and setting it to d inside the function.

In [13]:
def ways3(n,d,global_d=-1):
    if global_d<0:
        global_d=d
    if d<0 or d>global_d:
        return 0
    if n<0:
        return 0
    elif n==0 and d==0:
        return 1
    elif n==1 and d<=1:
        return 1
    if n<d:
        return 0
    else:
        return ways3(n-1,d,global_d)+ways3(n-1,d-1,global_d)+ways3(n-1,d+1,global_d)

In [14]:
ways3(8,5)

96